In [ ]:
from dotenv import load_dotenv

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone()

# Document Loader

In [10]:
pdf_loader = PyPDFLoader("../data/PT Bintang Toedjoe.pdf")
docs = pdf_loader.load()

print(f"Text : {docs[0].page_content[:200]}")
print(f"Metadata : {docs[0].metadata}")
print(f"Length : {len(docs[0].page_content)}")

Text : PT Bintang Toedjoe 
Vision & Mission 
We are Consumer Health Company Bintang Toedjoe 
PT. Bintang Toedjoe was founded in Garut, West Java, on April 29 1946 by Tan Jun She (a 
sinshe), Tjia Pu Tjien an
Metadata : {'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-03-04T20:35:42+07:00', 'author': 'Gemitry', 'moddate': '2025-03-04T20:35:42+07:00', 'source': '../data/PT Bintang Toedjoe.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}
Length : 2902


# Document Splitter

In [11]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs_splitted = splitter.split_documents(docs)

print(f"Text : {docs_splitted[0].page_content}")
print(f"Metadata : {docs_splitted[0].metadata}")
print(f"Length : {len(docs_splitted[0].page_content)}")

Text : PT Bintang Toedjoe 
Vision & Mission 
We are Consumer Health Company Bintang Toedjoe 
PT. Bintang Toedjoe was founded in Garut, West Java, on April 29 1946 by Tan Jun She (a 
sinshe), Tjia Pu Tjien and Hioe On Tjan. The name was chosen based on the number of 
Tan Jun She's daughters, namely seven daughters. With only traditional equipment and a 
few employees at that time, PT Bintang Toedjoe succeeded in producing over-the-counter
Metadata : {'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-03-04T20:35:42+07:00', 'author': 'Gemitry', 'moddate': '2025-03-04T20:35:42+07:00', 'source': '../data/PT Bintang Toedjoe.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}
Length : 434


# Vector Store

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from uuid import uuid4

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

vector_store = PineconeVectorStore(
    index_name="chatb7",
    embedding=embeddings,
)

uuids = [str(uuid4()) for _ in range(len(docs_splitted))]
vector_store.add_documents(documents=docs_splitted, ids=uuids)

['51e37468-69e6-46ce-be18-b8c59a99a70a',
 '5b89b9db-d0ab-451a-9081-4893f3a2c75f',
 '5bf67780-f2c1-4335-b785-908126b936cf',
 '5ac606f3-75de-4edc-bdff-ed718194409a',
 'bbed41e0-8e94-4556-a775-110b67229f0d',
 'c1ff4264-824a-4ee8-aa39-7762b56a0b8f',
 'a4c73016-5c74-4402-bcfc-710d2a8d82ae',
 '27ec841b-d255-4154-bc77-d94e3ea1eaec',
 'd240c693-1146-48bc-8a3d-262928cbb52b',
 'd7bf7fc2-a57e-4bb0-91d8-b0534a21b7cf',
 '16c4f245-6d7d-47ab-a8e3-53de74f06f3a',
 '532ee091-cc37-4c3b-a810-fa9d0dca5a25',
 '5c2cc9f4-2bcb-4e7b-b936-9d6efce85515',
 'e67cf95a-dca4-4b6f-be7b-ceadb12dc5f7',
 '63c89f9a-c3cc-4979-8c38-818866a3fcaf',
 'ef9d992e-0371-41de-bbe3-7c2826a7223c',
 '2437bb61-8d8c-4781-96da-6ef1c033e03a',
 'd145a24c-b1b7-45b7-9d08-68730b5301d7',
 '53ca3d25-44ae-4c24-a051-ec867763daa7',
 'ccb8ba8d-4344-475e-80f9-0f8027cf8b83',
 'ca0fc533-6430-4410-98c2-b805a40d87cf',
 '5697fffa-8f06-4f3b-8583-fb86b0137a7c',
 '936e4f64-4c2c-47e5-8916-c78cd5661501',
 '9516cab4-9bc2-4664-9345-acd8d6409dcb',
 '52c9f0a5-dd11-

In [25]:
results = vector_store.similarity_search(
    "Apa itu Bintang Toedjoe?",
    k=2,
)

for result in results:
    print(result.page_content, end="\n\n")

PT Bintang Toedjoe 
Vision & Mission 
We are Consumer Health Company Bintang Toedjoe 
PT. Bintang Toedjoe was founded in Garut, West Java, on April 29 1946 by Tan Jun She (a 
sinshe), Tjia Pu Tjien and Hioe On Tjan. The name was chosen based on the number of 
Tan Jun She's daughters, namely seven daughters. With only traditional equipment and a 
few employees at that time, PT Bintang Toedjoe succeeded in producing over-the-counter

actively in the company. Hopefully this award can be a motivation to continue to be better in the fu ture. 
Bintang Toedjoe… Better… Better… Better… Pasti Kelas Dunia!!  
Management Forum Quality Assurance and 
Regulatory Affairs (MFQARA Forum) 
19 December 2024 
Quality Assurance and Regulatory Affairs 
Awarded by Kalbe Corporate Manufacturing, Quality Assurance & Regulatory 
Affair 
Thursday, December 19, 2024 PT Bintang Toedjoe won the an award at the MFQARA Forum XV event held



# Chain

In [30]:
from langchain.chains.question_answering import load_qa_chain
from langchain_google_genai.llms import GoogleGenerativeAI
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Kamu adalah asisten AI yang membantu menjawab pertanyaan tentang PT Bintang Toedjoe menggunakan konteks yang diberikan.
Berikan jawaban yang lengkap, informatif, dan dalam Bahasa Indonesia yang natural dan ramah.

Konteks:
{context}

Pertanyaan:
{question}

Jawaban (dalam Bahasa Indonesia yang natural dan informatif):
"""

PROMPT = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["context", "question"],
)

llm = GoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05")
chain = load_qa_chain(llm, chain_type="stuff", prompt=PROMPT)

query = "Apa itu Extra Joss Laki Anggur?"
docs = vector_store.similarity_search(query, k=5)

result = chain.run(input_documents=docs, question=query)
display(result) 

'Extra Joss Laki Anggur adalah suplemen kesehatan yang diproduksi oleh PT Bintang Toedjoe. Suplemen ini memiliki manfaat untuk membantu menjaga stamina dan menyegarkan tubuh. Extra Joss Laki Anggur hadir dalam bentuk sachet dan memiliki varian rasa anggur.\n\nBerikut beberapa informasi penting mengenai Extra Joss Laki Anggur:\n\n*   **Indikasi:** Membantu menjaga stamina dan menyegarkan tubuh.\n*   **Dosis:** 1-3 kali sehari, 1 sachet setiap kali minum.\n*    **Varian:** Rasa Anggur\n*   **Kemasan:** Tersedia dalam kemasan berisi 6 sachet (@4g) dan 12 sachet (@4g).\n* **Komposisi:**\n    *   Kofein 50 mg\n    *   Ekstrak Ginseng Bubuk 35 mg\n    *   Vitamin B3 15 mg\n    *   Vitamin B6 2 mg\n    *   Vitamin B12 2.14 mcg\n    *   Royal Jelly 2 mg\n    *   Vitamin B5 5mg\n    *   Vitamin B2\n\n**Peringatan & Perhatian:**\n\n*   Tidak dianjurkan untuk penderita hipertensi dan diabetes.\n*   Tidak dianjurkan untuk anak-anak, wanita hamil, dan ibu menyusui.\n*   Harap ikuti anjuran pemakaia